Урок 4. Система управления базами данных MongoDB в Python
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).


In [1]:
# Импортируем нужные для работы библиотеки и функции:

from bs4 import BeautifulSoup
import requests
import re
import time
from re import sub
from decimal import Decimal
import io
from datetime import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
from pprint import pprint 
from pymongo.errors import DuplicateKeyError

Создаем базу данных для собранных вакансий:

In [2]:
clieent_TV = MongoClient()

In [3]:
db = clieent_TV['job_hh']

In [4]:
collections_vacancy_hh = db.hh

Парсер HH:

In [5]:
# Основа для предстоящей работы: основная ссылка и юзер-агент, чтобы не заблочили. 
# Пользовательский ввод:
user_requests = input('Введите запрос: ').strip() # Мой запрос Python
total_pages = int(input('Введите колличество страниц для сбора данных: ')) # Запрашиваю 4 стр.

url = 'https://krasnoznamensk.hh.ru/search/vacancy?text=' + user_requests+ '&salary=&ored_clusters=true&enable_snippets=true&page=0'

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',}

map = {} # Создаю пустой словарь и счетчик 
id = 0 # Для идентификации словаря внутри общего словоря (как id словоря)

start = time.time() # Функция для оценки времени

# Цикал для скрейпинга страниц
for page in range(total_pages): # парсить будем все страницы в указанном диапазоне 
    
    request_start_page = url + str(page) 

    print("Scraping page: %d" % (page +1)) # для вывода результата добавляем +1, чтобы первая страница была первой, а не 0

    # Мой основной запрос на скрейпинг всей страницы:
    my_requests = requests.get(request_start_page, headers=headers).text
    soup = BeautifulSoup(my_requests, 'html.parser')
    job_vacancy = soup.find_all("div", {"class": "vacancy-serp-item__layout"}) 
    
    print(len(job_vacancy))
    
     # Предохранитель(если что то не так с запросом, чтобы не случился "вечный двигатель"):

    page_nav = soup.find_all('a', {"class": "bloko-button"})

    if(len(page_nav) == 0):
        print("max page number: %d" % (page))
        end = time.time()
        print(end - start)
        break

    for vacancy in range(len(job_vacancy)):
        
        job = job_vacancy[vacancy]
        
        id += 1
        map[id] = {}
        
        # Сайт
        website = 'https://hh.ru/'

        #  Должность
        job_title_and_link = job.find('a', {"class": "serp-item__title"})
        
        #  Ссылка
        job_link = job_title_and_link['href']
        

        # Зарплата (обрабатываем пропуски, через регулярные выражения обрабатываем формат и тут же разносим данные по разным колонкам)
        if job.find('span', {"class": "bloko-header-section-3"}) is None:
            
            min_salary = 0 # 0 = "не указанна", что бы была возможность записать эту колонку int
            max_salary = 0 # 0 = "не указанна", что бы была возможность записать эту колонку int
            currency = 'Не указанно'
        else:
            salary = job.find('span', {"class": "bloko-header-section-3"}).text
            money = re.findall('[0-9]+\s?[0-9]+\s?[0-9]+', salary)
            currency = str(re.findall('\D+[^о]$', salary))
            for i in range(len(money)):
                if i == 0 :
                    min_salary = str(money[i]).replace('\u202f', '')
                elif i == 1:
                    max_salary = str(money[i]).replace('\u202f', '')
                    


        # Компания
        if job.find('a', {"class": "bloko-link bloko-link_kind-tertiary"}) is None:
            company = "Компания не указанна"
        else:
            company = job.find('a', {"class": "bloko-link bloko-link_kind-tertiary"}).text

        # Город
        if job.find('div',{"data-qa": "vacancy-serp__vacancy-address", "class": "bloko-text"}) is None:
            city = "Город не указанн"
        else:
            city = job.find('div',{"data-qa": "vacancy-serp__vacancy-address", "class": "bloko-text"}).text
        
        # Описание вакансии(сразу обрабатываем возможные пропуски)
        if job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_responsibility", "class": "bloko-text"}) is None:
            vacancy_description = "Описание отсутствует"
        else:
            vacancy_description = job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_responsibility", "class": "bloko-text"}).text
        
        #Требования к соискателю (сразу обрабатываем возможные пропуски)
        if job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_requirement", "class": "bloko-text"}) is None:
            requirements_for_the_applicant = "Требования к соискателю не указанны"
        else:
            requirements_for_the_applicant = job.find('div', {"data-qa": "vacancy-serp__vacancy_snippet_requirement", "class": "bloko-text"}).text

            
        #Записываем все найденное в словарь:
        map[id]["Сайт"] = str(website)
        map[id]["Должность"] = str(job_title_and_link)
        map[id]["Ссылка"] = str(job_link)
        map[id]["Минимальная заработная плата"] = int(min_salary)
        map[id]["Максимальная заработная плата"] = int(max_salary)
        map[id]["Валюта"] = str(currency)
        map[id]["Компания"] = str(company)
        map[id]["Город"] = str(city)
        map[id]["Описание вакансии"] = str(vacancy_description)
        map[id]["Требования к соискателю"] = str(requirements_for_the_applicant)
        
        # Создаем проверку. Сначала смотрим нет ли таких вакансий в нашей бд, 
        # только после добавляем. Уникальность проверяем по ссылке на вакансию
        
        if not len(list(collections_vacancy_hh.find({"Ссылка": job_link}))):
            db.hh.insert_one(map[id])
        else:
            print(f'Эта вакансия уже есть в базе данных!\n{map[id]}')

        

#result = map



Введите запрос: Python
Введите колличество страниц для сбора данных: 4
Scraping page: 1
20
Scraping page: 2
20
Scraping page: 3
20
Эта вакансия уже есть в базе данных!
{'Сайт': 'https://hh.ru/', 'Должность': '<a class="serp-item__title" data-qa="serp-item__title" href="https://krasnoznamensk.hh.ru/vacancy/77397988?from=vacancy_search_list&amp;query=Python" target="_blank">Программист Python\\Django</a>', 'Ссылка': 'https://krasnoznamensk.hh.ru/vacancy/77397988?from=vacancy_search_list&query=Python', 'Минимальная заработная плата': 0, 'Максимальная заработная плата': 0, 'Валюта': 'Не указанно', 'Компания': 'ООО\xa0ИнфоДев', 'Город': 'Хабаровск', 'Описание вакансии': 'Разработка веб-приложений с использованием Python\\Django. Проектирование и реализация REST-сервисов. Проектирование и реализация архитектуры и бизнес-логики системы. ', 'Требования к соискателю': '2-3 года опыта коммерческой разработки веб-приложений на Python>=2.7 и Django >= 1.9. Опыт работы с Django...'}
Эта вакансия уж

Сразу видим, что проверка на уникальность работает, из собранных 4 страниц 4 вакансии оказались не уникальными, они дублируются на hh

In [6]:
for document in db.hh.find(): # выводим эти вакансии 
    pprint(document)

{'_id': ObjectId('63fcfbcddab975b70588f72e'),
 'Валюта': 'Не указанно',
 'Город': 'Москва',
 'Должность': '<a class="serp-item__title" data-qa="serp-item__title" '
              'href="https://krasnoznamensk.hh.ru/vacancy/69166398?from=vacancy_search_list&amp;query=Python" '
              'target="_blank">Аналитик данных / Data Analyst</a>',
 'Компания': 'Сбер. Экспертам и руководителям',
 'Максимальная заработная плата': 0,
 'Минимальная заработная плата': 0,
 'Описание вакансии': 'Проверять продуктовые гипотезы и готовить Ad-hoc '
                      'аналитику для поддержки принятия бизнес-решений. '
                      'Проводить A/B тесты и интерпретировать результаты. ',
 'Сайт': 'https://hh.ru/',
 'Ссылка': 'https://krasnoznamensk.hh.ru/vacancy/69166398?from=vacancy_search_list&query=Python',
 'Требования к соискателю': 'Опыт работы с большими массивами данных. Знания '
                            'SQL и Python. Опыт создания дашбордов в Tableau '
                           

Запрашиваю на вывод вакансии с зп больше 80 000, и указываю что это должны быть рубли так как у меня в бд есть и другая валюта:

In [12]:
for document in db.hh.find({'Минимальная заработная плата': {'$gt': 80000},
                           'Максимальная заработная плата': {'$gt': 80000},
                           'Валюта': "[' руб.']"}):
    pprint(document)

{'_id': ObjectId('63fcfbcddab975b70588f734'),
 'Валюта': "[' руб.']",
 'Город': 'Санкт-Петербург',
 'Должность': '<a class="serp-item__title" data-qa="serp-item__title" '
              'href="https://krasnoznamensk.hh.ru/vacancy/77091466?from=vacancy_search_list&amp;query=Python" '
              'target="_blank">Разработчик Python</a>',
 'Компания': 'ООО\xa0Нетопия',
 'Максимальная заработная плата': 250000,
 'Минимальная заработная плата': 200000,
 'Описание вакансии': 'Работка backend части программного обеспечения. '
                      'Разработка внутренних библиотек. Оптимизация узких мест '
                      'в основной архитектуре продукта.',
 'Сайт': 'https://hh.ru/',
 'Ссылка': 'https://krasnoznamensk.hh.ru/vacancy/77091466?from=vacancy_search_list&query=Python',
 'Требования к соискателю': 'Владение Python (asyncio, aiohttp, pydantic, '
                            'FastAPI, sqlalchemy). Понимание зачем нужен ООП. '
                            'Умение работать с GIT. По